In [229]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns

In [231]:
df = pd.read_csv("/Users/matteo/Documents/PersonalProjects/airbnb-pricing-optimization/data/listings-1.csv")

# check for NAs
df.isna().sum()

id                                   0
name                                 0
host_id                              0
host_name                            1
neighbourhood_group               7780
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                             1985
minimum_nights                       0
number_of_reviews                    0
last_review                       1870
reviews_per_month                 1870
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
license                           2647
dtype: int64

In [233]:
# save the observations with missing prices
df_missing_prices = df[df["price"].isna()]

# remove entries with missing prices
df = df[df["price"].isna() == False]

# define price outliers
Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)

upper_bound = Q3 + (1.5 * (Q3 - Q1))

df["is_outlier"] = (df["price"] > upper_bound)

# drop neighbourhood_group, as all entries have NAs
df = df.drop(columns = "neighbourhood_group")

# change all NA's in license to "None" because NA means that no license is present
df["license"] = df["license"].fillna("None")

# remove listing with no host name
df = df[df["host_name"].isna() == False]

In [235]:
df.isna().sum()

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       1153
reviews_per_month                 1153
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
license                              0
is_outlier                           0
dtype: int64

In [237]:
df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,is_outlier
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.770280,-122.433170,Entire home/apt,157.0,2,496,2025-08-16,2.53,1,224,41,STR-0006854,False
1,5858,Creative Sanctuary,8904,Philip Jonathon,Bernal Heights,37.744740,-122.420890,Entire home/apt,250.0,30,105,2017-08-06,0.53,1,357,0,None,False
2,8014,female HOST quiet fast internet market parking,22402,Jia,Outer Mission,37.730770,-122.448270,Private room,67.0,4,90,2025-07-03,0.57,3,42,1,STR-0000974,False
3,8142,*FriendlyRoom Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.765550,-122.452130,Private room,56.0,32,10,2023-07-30,0.07,20,362,0,None,False
4,8339,Historic Alamo Square Victorian,24215,Rosmarie,Western Addition,37.773770,-122.436140,Entire home/apt,527.0,9,25,2019-06-28,0.13,1,327,0,STR-0000264,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7775,1498890967225433381,Golden hour escape /Pink house near beach & GG...,182770575,Sunny,Outer Sunset,37.748388,-122.489829,Entire home/apt,333.0,2,0,NaN,NaN,2,149,0,STR-0006775,False
7776,1498958112086931823,"House w/2 king beds, private baths, and a hot tub",9194713,Lance,Visitacion Valley,37.719620,-122.402330,Private room,168.0,1,0,NaN,NaN,4,108,0,2025-005560STR,False
7777,1499028835988470211,Bedroom with a private roof deck and a great view,3107918,Minh,Ocean View,37.712670,-122.468830,Private room,46.0,30,0,NaN,NaN,2,365,0,None,False
7778,1499581470510980610,Chic & Plant filled apartment in Russian Hill,98894458,Tulika,Nob Hill,37.794540,-122.415760,Private room,90.0,31,0,NaN,NaN,2,320,0,None,False
